In [8]:
import pandas as pd
import numpy as np
import os
import scipy
import sqlite3
import tensorflow as tf
import string

In [2]:
curr_path = os.path.realpath('')
print(curr_path)
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
cur = conn.cursor()
cur.execute("select * from loan limit 5;")
results = cur.fetchall()
# print(results)
cur.close()
conn.close()

/Users/noellin/Jupyter Notebooks/loan_driving2


In [3]:
conn = sqlite3.connect("lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status = 'Charged Off';", conn)
#print(df)
conn.close()

In [4]:
ddict = df.to_dict()
print(ddict['loan_amnt'][7])
ddict['sub_grade'][7]
g = zip(ddict['loan_amnt'].values(),ddict['sub_grade'].values(),ddict['id'].values(),ddict['loan_status'].values())
print(len(list(g)))

15000.0
45248


In [5]:
# create our data set
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
# df = pd.read_sql_query("select loan_amnt,loan_status,sub_grade from loan where loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)';", conn)
#df_tf = pd.read_sql_query("select loan_amnt, term, int_rate, annual_inc, dti, home_ownership from loan where loan_status =='Fully Paid' OR loan_status == 'Charged Off' OR loan_status == 'Default'", conn)
df_tf = pd.read_sql_query("select loan_status, term, int_rate, emp_length, home_ownership, verification_status, loan_amnt, annual_inc, dti from loan where emp_length != 'n/a' OR loan_status =='Fully Paid' OR loan_status == 'Charged Off' OR loan_status == 'Default'", conn)
# df_desc = pd.read_sql_query("select loan_amnt, loan_status, sub_grade from loan where desc != 'NULL' AND loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)'", conn)
print (len(df_tf))

data = []
results = []
for i in range(10000):  # data set of 80000 for faster run time. Replace with range(len(df_tf)) later
    # one hot encoding for loan status. Or 0, 1, 2
    # loan_status = ['1', '0', '0'] if df_tf.iloc[i][0] == 'Fully Paid' else ['0', '1', '0'] if df_tf.iloc[i][0] == 'Default' else ['0', '0', '1']
    loan_status = '0' if df_tf.iloc[i][0] == 'Fully Paid' else '1' if df_tf.iloc[i][0] == 'Default' else '2'
    
    # 0 if term is 36 months, 1 if 60 months
    term = 0 if df_tf.iloc[i][1] == ' 36 months' else 1
    
    # change interest rate from str to float
    int_rate = float(df_tf.iloc[i][2].replace('%', ''))
    
    # get employment length
    txt = ''.join([c for c in df_tf.iloc[i][3].lower() if not c in set(string.punctuation)])
    emp_length_list = ([int(s) for s in txt.split() if s.isdigit()])
    emp_length = 0 if emp_length_list == [] else emp_length_list[0]

    # 0 if renting, 1 if owner
    ownership = 0 if df_tf.iloc[i][4] == 'RENT' else 1
    
    # verification status, 0 for not verified, 1 for source verified and 2 for verified
    verification = 0 if df_tf.iloc[i][5] == 'Not Verified' else 1 if df_tf.iloc[i][5] == 'Source Verified' else 2

    # append in feature vector
    data.append([term, int_rate, emp_length, ownership, verification, df_tf.iloc[i][6], df_tf.iloc[i][7], df_tf.iloc[i][8]])
                 
    # add loan status to results 
    results.append(loan_status)

print (data[-1:])
X_train = data[:int(len(data)/2)]
y_train = results[:int(len(results)/2)]
X_val = data[int(len(data)/2): int(len(data)*0.75)]
y_val = results[int(len(results)/2): int(len(results)*0.75)]
X_test = data[int(len(data)*.75):]
y_test = results[int(len(results)*.75):]

conn.close()
# feature vector - predictor will predict ("charged off/default status") vs. 
# [1(maybe check this),'sub_grade','loan_amnt','emp_length','home_ownership','annual_inc','verification_status','dti'(debt-to-income),'']

852525
[[1, 13.99, 8, 1, 2, 28000.0, 200000.0, 9.6999999999999993]]


NameError: name 'numpy' is not defined

In [9]:
X_train_arr = np.array(X_train)
y_train_arr = np.array(y_train)
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train_arr)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[200,100], n_classes=3,feature_columns=feature_cols, label_keys=['0','1','2'])
# dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) 
dnn_clf.fit(X_train_arr, y_train_arr, batch_size=10, steps=5000)
# dnn_clf.fit(X_train_arr, y_train_arr)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11901a5c0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving i

/Users/noellin/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/cv/j7y0jf0x0gbc39xntlnng44c0000gn/T/tmpaygqmg32/model.ckpt.
INFO:tensorflow:loss = 8818.46, step = 1
INFO:tensorflow:global_step/sec: 636.01
INFO:tensorflow:loss = 36.8473, step = 101 (0.164 sec)
INFO:tensorflow:global_step/sec: 433.121
INFO:tensorflow:loss = 9.42107, step = 201 (0.227 sec)
INFO:tensorflow:global_step/sec: 653.796
INFO:tensorflow:loss = 9.53088, step = 301 (0.153 sec)
INFO:tensorflow:global_step/sec: 347.8
INFO:tensorflow:loss = 2.41896, step = 401 (0.286 sec)
INFO:tensorflow:global_step/sec: 888.257
INFO:tensorflow:loss = 1.3975, step = 501 (0.113 sec)
INFO:tensorflow:global_step/sec: 846.84
INFO:tensorflow:loss = 1.27981, step = 601 (0.118 sec)
INFO:tensorflow:global_step/sec: 818.198
INFO:tensorflow:loss = 1.17604, step = 701 (0.122 sec)
INFO:tensorflow:global_step/sec: 550.427
INFO:tensorflow:loss = 0.702641, step = 801 (0.182 sec)
INFO:tensorflow:global_step/sec:

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x10efe6668>, 'hidden_units': [200, 100], 'feature_columns': (_RealValuedColumn(column_name='', dimension=8, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x10db20510>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [11]:
from sklearn.metrics import accuracy_score

X_val_arr = np.array(X_val)
y_val_arr = np.array(y_val)
y_pred = dnn_clf.predict_classes(X_val_arr, as_iterable=False)
#print (X_val_arr)
#print (X_val[0])
#print(y_pred)
print (accuracy_score(y_val_arr, y_pred))

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


/Users/noellin/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Restoring parameters from /var/folders/cv/j7y0jf0x0gbc39xntlnng44c0000gn/T/tmpaygqmg32/model.ckpt-5000


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead.

In [1]:
X_train_arr = numpy.array(X_train)
y_train_arr = numpy.array(y_train)

print (y_train_arr[:5])
print (y_train[:5])
print (X_train_arr[:5])

NameError: name 'numpy' is not defined

In [ ]:
# for testing library purposes
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
print (cancer['data'])
print (cancer['target'])